# This is the main analysis script done for Assignment #1 

Following analysis is done: 

1. Evaluate the initial performance of following models: DecisionTree, KNN, SVM under default dataset in terms of accuracy & speed 
2. Evaluate the performance of the model in terms of sample size & features (less features + more noise in features) 

In [ ]:
import pandas as pd 
import numpy as np 
import os 
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

from IPython.display import display 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV

In [1]:
from utils import ml_dataset, classificationModel

<class 'ModuleNotFoundError'>: No module named 'utils'

# Part 1：Reading and Prepping dataset 

In [4]:
credit = pd.read_csv('/drive/notebooks/credit_risk_dataset.csv')

In [5]:
credit_df = ml_dataset(credit, 'loan_status') 
# get a summary of the dataset 
credit_df.summary()
# fill in null values appropriately 
credit_df.fill_null()

# split into train-test dataset 
X_train, X_test, y_train, y_test = credit_df.training_prep()


---------------------- Basic Attributes -------------------------


,PctNull,DataTypes
person_age,0.00,int64
person_income,0.00,int64
person_home_ownership,0.00,object
person_emp_length,0.03,float64
loan_intent,0.00,object
loan_grade,0.00,object
loan_amnt,0.00,int64
loan_int_rate,0.10,float64
loan_status,0.00,int64
loan_percent_income,0.00,float64


----------------- Distribution of Continuous Variable -------------------


,PctNull,DataTypes,count,mean,std,min,25%,50%,75%,max
person_age,0.00,int64,32581.0,27.734600,6.348078,20.00,23.00,26.00,30.00,144.00
person_income,0.00,int64,32581.0,66074.848470,61983.119168,4000.00,38500.00,55000.00,79200.00,6000000.00
person_emp_length,0.03,float64,31686.0,4.789686,4.142630,0.00,2.00,4.00,7.00,123.00
loan_amnt,0.00,int64,32581.0,9589.371106,6322.086646,500.00,5000.00,8000.00,12200.00,35000.00
loan_int_rate,0.10,float64,29465.0,11.011695,3.240459,5.42,7.90,10.99,13.47,23.22
loan_status,0.00,int64,32581.0,0.218164,0.413006,0.00,0.00,0.00,0.00,1.00
loan_percent_income,0.00,float64,32581.0,0.170203,0.106782,0.00,0.09,0.15,0.23,0.83
cb_person_cred_hist_length,0.00,int64,32581.0,5.804211,4.055001,2.00,3.00,4.00,8.00,30.00


----------------- Corrlation matrix -----------------------------------


...

person_home_ownership loan_status          
                              count      mean
0              MORTGAGE       13444  0.125707
1                 OTHER         107  0.308411
2                   OWN        2584  0.074690
3                  RENT       16446  0.315700

loan_intent loan_status          
                           count      mean
0  DEBTCONSOLIDATION        5212  0.285879
1          EDUCATION        6453  0.172168
2    HOMEIMPROVEMENT        3605  0.261026
3            MEDICAL        6071  0.267007
4           PERSONAL        5521  0.198877
5            VENTURE        5719  0.148103

loan_grade loan_status          
                   count      mean
0          A       10777  0.099564
1          B       10451  0.162760
2          C        6458  0.207340
3          D        3626  0.590458
4          E         964  0.644191
5          F         241  0.705394
6          G          64  0.984375

cb_person_default_on_file loan_status          
                                  count      mean
0                         N       26836  0.183932
1                         Y        5745  0.378068

,PctNull,DataTypes
person_age,0.00,int64
person_income,0.00,int64
person_home_ownership,0.00,object
person_emp_length,0.03,float64
loan_intent,0.00,object
loan_grade,0.00,object
loan_amnt,0.00,int64
loan_int_rate,0.10,float64
loan_status,0.00,int64
loan_percent_income,0.00,float64


# Part 2: Understand model performance with vs. w.o hyperparameter tuning 

## Part 2.1 Default performance - No hyperparameter tuning 

In [6]:
modelType = ['DecisionTree', 'KNN', 'SVM', 'NeuralNetwork']
modelObjectLst = [] 
defaultPerformance = pd.DataFrame() 

for model in modelType: 
    modelObject = classificationModel(model) 
    modelObjectLst.append(modelObject) 
    modelObject.fit(X_train, y_train) 
    performance = modelObject.evaluate_performance(X_train, y_train, X_test, y_test)
    performance['modelType'] = model 
    defaultPerformance = pd.concat([defaultPerformance, performance], axis = 0)


defaultPerformance = defaultPerformance[['modelType', 'sample', 'accuracy', 'auc', 'time (seconds)']]
defaultPerformance.to_csv('/drive/notebooks/defaultPerformance.csv')

<class 'NameError'>: name 'MLPClassifier' is not defined

In [14]:

defaultPerformance 

,modelType,sample,accuracy,auc,time (seconds)
0,DecisionTree,Train,1.000000,1.000000,0.129
1,DecisionTree,Test,0.883785,0.839615,0.003
0,KNN,Train,0.915154,0.961129,0.004
1,KNN,Test,0.888696,0.863768,2.555
0,SVM,Train,0.899325,0.893269,72.850
1,SVM,Test,0.895038,0.884671,5.022


## Part 2.2 Performance - With some hyperparameter tuning 

In [20]:
modelType = ['DecisionTree', 'KNN', 'SVM']
modelObjectLst = [] 
tunedPerformance = pd.DataFrame() 

for model in modelType: 
    modelObject = classificationModel(model) 
    modelObjectLst.append(modelObject) 
    modelObject.GridSearch(X_train, y_train) 
    performance = modelObject.evaluate_performance(X_train, y_train, X_test, y_test)
    performance['modelType'] = model 
    performance['bestParam'] = str(modelObject.GridSearchCV.best_params_)
    tunedPerformance = pd.concat([tunedPerformance, performance], axis = 0)

tunedPerformance = tunedPerformance[['modelType', 'bestParam', 'sample', 'accuracy', 'auc', 'time (seconds)']]
tunedPerformance.to_csv('/drive/notebooks/tunedPerformance.csv')

performing hyperparameter tuning for Decision Tree
best parameter:
{'max_depth': 10, 'max_features': 23}
performing hyperparameter tuning for KNN
best parameter:
{'n_neighbors': 10, 'weights': 'distance'}
performing hyperparameter tuning for SVM
best parameter:
{'degree': 2}


In [21]:
tunedPerformance

,modelType,bestParam,sample,accuracy,auc,time (seconds)
0,DecisionTree,"{'max_depth': 10, 'max_features': 23}",Train,0.935236,0.926854,0.000
1,DecisionTree,"{'max_depth': 10, 'max_features': 23}",Test,0.922455,0.900036,0.002
0,KNN,"{'n_neighbors': 10, 'weights': 'distance'}",Train,1.000000,1.000000,0.000
1,KNN,"{'n_neighbors': 10, 'weights': 'distance'}",Test,0.894834,0.881782,2.565
0,SVM,{'degree': 2},Train,0.899325,0.893261,0.000
1,SVM,{'degree': 2},Test,0.895038,0.884677,4.748


## Part 3: Simulate different X_Trains under different sample size and different # of features to see the robustness of model performance 

Following scenarios are simulated: 
- sample size: 20%, 40%, 60%, 80% of current X_train 
- features: There are overall 23 features, randomly take 5, 10, 15 of the features

For each sceanrio, tune hyperparameter and store model performance 

In [42]:
modelType = ['DecisionTree', 'KNN', 'SVM']
modelObjectLst = [] 
tunedPerformance = pd.DataFrame() 

for nrow in [0.05, 0.1, 0.2, 0.4, 0.6, 0.8]: 
    for ncol in [5, 10, 15]: 
        print('sampling nrow {} and ncol {}'.format(nrow, ncol))
        # first do row wise sampling for X_train and y_train 
        X_train_rowSampled = X_train.sample(frac = nrow, axis = 0, random_state = 42) 
        y_train_sampled = y_train.sample(frac = nrow, axis = 0, random_state = 42) 
        # then do X_train column sampling specifically 
        X_train_sampled = X_train_rowSampled.sample(n = ncol, axis = 1, random_state = 42) 
        SampleSize = X_train_sampled.shape[0]
        numFeature = X_train_sampled.shape[1] 
        
        for model in modelType: 
            modelObject = classificationModel(model) 
            modelObjectLst.append(modelObject) 
            modelObject.GridSearch(X_train_sampled, y_train_sampled) 
            performance = modelObject.evaluate_performance(X_train_sampled, y_train_sampled, X_test[X_train_sampled.columns], y_test)
            performance['modelType'] = model 
            performance['SampleSize'] = SampleSize 
            performance['NumFeature'] = numFeature 
            performance['bestParam'] = str(modelObject.GridSearchCV.best_params_)
            tunedPerformance = pd.concat([tunedPerformance, performance], axis = 0)
            
tunedPerformance.to_csv('/drive/notebooks/tunedPerformanceSampled.csv')

sampling nrow 0.05 and ncol 5
performing hyperparameter tuning for Decision Tree
best parameter:
{'max_depth': 5, 'max_features': 10}
performing hyperparameter tuning for KNN
best parameter:
{'n_neighbors': 20, 'weights': 'uniform'}
performing hyperparameter tuning for SVM
best parameter:
{'degree': 2}
sampling nrow 0.05 and ncol 10
performing hyperparameter tuning for Decision Tree
best parameter:
{'max_depth': 5, 'max_features': 20}
performing hyperparameter tuning for KNN
best parameter:
{'n_neighbors': 20, 'weights': 'distance'}
performing hyperparameter tuning for SVM
best parameter:
{'degree': 2}
sampling nrow 0.05 and ncol 15
performing hyperparameter tuning for Decision Tree
best parameter:
{'max_depth': 5, 'max_features': 23}
performing hyperparameter tuning for KNN
best parameter:
{'n_neighbors': 10, 'weights': 'distance'}
performing hyperparameter tuning for SVM
best parameter:
{'degree': 2}
sampling nrow 0.1 and ncol 5
performing hyperparameter tuning for Decision Tree
best

In [11]:
tunedPerformance = pd.read_csv('/drive/notebooks/tunedPerformanceSampled.csv')
tunedPerformance.head()
tunedPerformance[tunedPerformance['sample'] == 'Test'].pivot(index = ['modelType', 'SampleSize'], columns = ['NumFeature'], values = 'auc')
                       

NumFeature                     5         10        15
modelType    SampleSize                              
DecisionTree 1140        0.549468  0.761092  0.825433
             2281        0.570296  0.773870  0.843206
             4561        0.567149  0.764330  0.856185
             9122        0.569157  0.773248  0.869782
             13684       0.569444  0.777358  0.870420
             18245       0.571669  0.770685  0.869036
KNN          1140        0.552835  0.731827  0.824705
             2281        0.557728  0.710620  0.844645
             4561        0.554413  0.734366  0.853106
             9122        0.551954  0.743387  0.863958
             13684       0.555914  0.750700  0.872950
             18245       0.546357  0.751055  0.883203
SVM          1140        0.494446  0.729569  0.857758
             2281        0.531837  0.730636  0.867193
             4561        0.559165  0.739498  0.873093
             9122        0.491717  0.739063  0.878671
             13684       0.549314  0.740376  0.879944
             18245       0.530198  0.739255  0.881387

## Part 4: Check how sensitive each model is to incoming noise in the feature sets 

create a function to randomly create features following a uniform or normal distribution 

or randomly create a bernouli distribution with certain success probability 

In [6]:
def generate_scaled_variates(distribution_func, size, random_seed = 42, *args, **kwargs): 
    np.random.seed(random_seed) 
    samples = distribution_func(size = size, *args,  **kwargs) 
    # min-max scale within 0 and 1 
    scaled_sample = (samples - np.min(samples)) / (np.max(samples) - np.min(samples)) 

    return scaled_sample 

randNormal = generate_scaled_variates(np.random.normal, loc = 0, size = 1000, scale = 1) 
randUniform = generate_scaled_variates(np.random.random, size = 1000) 
randExponential = generate_scaled_variates(np.random.exponential, size = 1000)
randBinomial =  generate_scaled_variates(np.random.random, size = 1000) < 0.1  

# Create two datasets containing the noise of X_train and X_test 
# This dataset will contain a random normal, unitform, exponential and binary variable with probability 10%, 20%, 30%, 40%, 50%
def generate_noise_dataset(X): 
    
    df_noise = pd.DataFrame({'randNormal': generate_scaled_variates(np.random.normal, loc = 0, size = len(X), scale = 1),
                             'randUniform': generate_scaled_variates(np.random.random, size = len(X)), 
                             'randExponential': generate_scaled_variates(np.random.exponential, size = len(X)), 
                             'randBinary10': generate_scaled_variates(np.random.random, size = len(X), random_seed = 43) < 0.1, 
                             'randBinary20': generate_scaled_variates(np.random.random, size = len(X), random_seed = 44) < 0.2, 
                             'randBinary30': generate_scaled_variates(np.random.random, size = len(X), random_seed = 45) < 0.3,
                             'randBinary40': generate_scaled_variates(np.random.random, size = len(X), random_seed = 46) < 0.4,
                            })
    return df_noise 

X_train_noise = generate_noise_dataset(X_train) 
X_test_noise = generate_noise_dataset(X_test)

In [1]:
modelType = ['DecisionTree', 'KNN']
modelObjectLst = [] 
tunedPerformance = pd.DataFrame() 

for n_noise in [0, 1, 2, 3, 4, 5, 6, 7]: 
    print('adding {} noisy features'.format(n_noise))
    # sample n noisy features from X_train  
    X_train_noise_sampled = X_train_noise.sample(n = n_noise, axis = 1, random_state = 42) 
    X_train_w_noise = pd.concat([X_train.reset_index(), X_train_noise_sampled.reset_index()], axis = 1) 
    
    X_test_noise_sampled = X_test_noise.sample(n = n_noise, axis = 1, random_state = 42) 
    X_test_w_noise = pd.concat([X_test.reset_index(), X_test_noise_sampled.reset_index()], axis = 1) 
    
    for model in modelType: 
        modelObject = classificationModel(model) 
        modelObjectLst.append(modelObject) 
        modelObject.GridSearch(X_train_w_noise, y_train) 
        performance = modelObject.evaluate_performance(X_train_w_noise, y_train, X_test_w_noise, y_test)
        performance['modelType'] = model 
        performance['numNoise'] = n_noise 
        performance['bestParam'] = str(modelObject.GridSearchCV.best_params_)
        tunedPerformance = pd.concat([tunedPerformance, performance], axis = 0)
            
tunedPerformance.to_csv('/drive/notebooks/tunedPerformanceWithNoise.csv')


<class 'NameError'>: name 'pd' is not defined

In [16]:
tunedPerformance[tunedPerformance['sample'] == 'Test'].pivot(index = ['modelType'], columns = ['numNoise'], values = 'auc')



numNoise,1,2,3,4,5,6,7
modelType,,,,,,,
DecisionTree,0.892113,0.894589,0.895234,0.896717,0.891943,0.884996,0.867435
KNN,0.570872,0.570883,0.570894,0.570894,0.570894,0.570894,0.570883
